In [11]:
%load_ext autoreload
%autoreload 2
from pathlib import Path
import shutil
import thor.data as data
import thor.data.dispatch as dispatch
import thor.grid as grid
import thor.track as track
import thor.option as option
import thor.visualize as visualize

notebook_name = "cpol_demo.ipynb"

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [27]:
# Parent directory for saving outputs
base_local = Path.home() / "THOR_output"
start = "2005-11-13T16:00:00"
end = "2005-11-13T18:00:00"

output_directory = base_local / "runs/cpol_demo_geographic"
options_directory = output_directory / "options"

if output_directory.exists():
    shutil.rmtree(output_directory / "options")

# Create the data_options dictionary
converted_options = {"save": True, "load": False, "parent_converted": None}
cpol_options = data.aura.cpol_data_options(
    start=start, end=end, converted_options=converted_options
)
# Restrict the ERA5 data to a smaller region containing the CPOL radar
lon_range = [129, 133]
lat_range = [-14, -10]
era5_pl_options = data.era5.data_options(
    start=start, end=end, latitude_range=lat_range, longitude_range=lon_range
)
era5_sl_options = data.era5.data_options(
    start=start,
    end=end,
    data_format="single-levels",
    latitude_range=lat_range,
    longitude_range=lon_range,
)
data_options = option.consolidate_options(
    [cpol_options, era5_pl_options, era5_sl_options]
)

dispatch.check_data_options(data_options)
data.option.save_data_options(data_options, options_directory)

grid_options = grid.create_options(name="geographic")
grid.check_options(grid_options)
grid.save_grid_options(grid_options, options_directory)

# Create the track_options dictionary
track_options = option.mcs(dataset="cpol")
option.save_track_options(track_options, options_directory)

# Create the display_options dictionary
visualize_options = {
    obj: visualize.option.runtime_options(obj, save=True, style="presentation")
    for obj in ["mcs"]
}
# visualize_options["middle_echo"] = visualize.option.runtime_options(
#     "middle_echo", save=True, style="presentation", figure_types=["mask"]
# )
# visualize.option.save_display_options(visualize_options, options_directory)
visualize_options = None

2024-09-28 12:42:21,534 - thor.option - DEBUG - Saving options to /home/ewan/THOR_output/runs/cpol_demo_geographic/options/data.yml
2024-09-28 12:42:21,578 - thor.grid - WARNING - Shape not specified. Will attempt to infer from input.
2024-09-28 12:42:21,580 - thor.grid - WARNING - Coordinates not specified. Will attempt to infer from input.
2024-09-28 12:42:21,582 - thor.option - DEBUG - Saving options to /home/ewan/THOR_output/runs/cpol_demo_geographic/options/grid.yml
2024-09-28 12:42:21,613 - thor.option - DEBUG - Saving options to /home/ewan/THOR_output/runs/cpol_demo_geographic/options/track.yml


In [37]:
if (output_directory / "attributes").exists():
    shutil.rmtree(output_directory / "attributes")
if (output_directory / "masks").exists():
    shutil.rmtree(output_directory / "masks")
times = data.utils.generate_times(data_options["cpol"])
tracks = track.simultaneous_track(
    times,
    data_options,
    grid_options,
    track_options,
    visualize_options,
    output_directory=output_directory
)

2024-09-28 13:46:48,804 - thor.track - INFO - Beginning thor run. Saving output to /home/ewan/THOR_output/runs/cpol_demo_geographic.
2024-09-28 13:46:48,805 - thor.track - INFO - Beginning simultaneous tracking.
2024-09-28 13:46:48,874 - thor.track - INFO - Processing 2005-11-13T16:00:08.
2024-09-28 13:46:48,876 - thor.data.aura - INFO - Updating cpol dataset for 2005-11-13T16:00:08.
2024-09-28 13:46:48,878 - thor.data.aura - INFO - Converting cpol data from twp10cpolgrid150.b2.20051113.160000.nc
2024-09-28 13:46:49,408 - thor.track - INFO - Processing hierarchy level 0.
2024-09-28 13:46:49,409 - thor.track - INFO - Tracking cell.
2024-09-28 13:46:49,435 - thor.match.match - INFO - Matching cell objects.
2024-09-28 13:46:49,436 - thor.match.match - INFO - No previous mask, or no objects in previous mask.
2024-09-28 13:46:49,439 - thor.visualize.runtime - INFO - Generating runtime visualizations.
2024-09-28 13:46:49,439 - thor.track - INFO - Tracking middle_echo.
2024-09-28 13:46:49,448

# Cartesian Coordinates

In [28]:
output_directory = base_local / "runs/cpol_demo_cartesian"
options_directory = output_directory / "options"

if (output_directory / "options").exists():
    shutil.rmtree(output_directory / "options")

grid_options = grid.create_options(name="cartesian", regrid=False)
grid.check_options(grid_options)
grid.save_grid_options(grid_options, options_directory)
data.option.save_data_options(data_options, options_directory)
option.save_track_options(track_options, options_directory)
# visualize.option.save_display_options(visualize_options, options_directory)


2024-09-27 20:22:15,494 - thor.grid - WARNING - Shape not specified. Will attempt to infer from input.
2024-09-27 20:22:15,495 - thor.option - DEBUG - Saving options to /home/ewan/THOR_output/runs/cpol_demo_cartesian/options/grid.yml
2024-09-27 20:22:15,500 - thor.option - DEBUG - Saving options to /home/ewan/THOR_output/runs/cpol_demo_cartesian/options/data.yml
2024-09-27 20:22:15,543 - thor.option - DEBUG - Saving options to /home/ewan/THOR_output/runs/cpol_demo_cartesian/options/track.yml


In [29]:
if output_directory.exists():
    shutil.rmtree(output_directory / "attributes")
    shutil.rmtree(output_directory / "masks")
times = data.utils.generate_times(data_options["cpol"])
tracks = track.simultaneous_track(
    times,
    data_options,
    grid_options,
    track_options,
    visualize_options,
    output_directory=output_directory,
)

2024-09-27 20:22:16,523 - thor.track - INFO - Beginning thor run. Saving output to /home/ewan/THOR_output/runs/cpol_demo_cartesian.
2024-09-27 20:22:16,524 - thor.track - INFO - Beginning simultaneous tracking.
2024-09-27 20:22:16,591 - thor.track - INFO - Processing 2005-11-13T16:00:08.
2024-09-27 20:22:16,592 - thor.data.aura - INFO - Updating cpol dataset for 2005-11-13T16:00:08.
2024-09-27 20:22:16,593 - thor.data.aura - INFO - Converting cpol data from twp10cpolgrid150.b2.20051113.160000.nc
2024-09-27 20:22:16,719 - thor.track - INFO - Processing hierarchy level 0.
2024-09-27 20:22:16,721 - thor.track - INFO - Tracking cell.
2024-09-27 20:22:16,757 - thor.match.match - INFO - Matching cell objects.
2024-09-27 20:22:16,759 - thor.match.match - INFO - No previous mask, or no objects in previous mask.
2024-09-27 20:22:16,768 - thor.visualize.runtime - INFO - Generating runtime visualizations.
2024-09-27 20:22:16,769 - thor.track - INFO - Tracking middle_echo.
2024-09-27 20:22:16,787 

# Analysis

In [38]:
import thor.analyze as analyze

In [58]:
analysis_options = analyze.mcs.analysis_options()
analyze.mcs.process_velocities(output_directory)

2024-09-30 10:09:24,265 - thor.write.attribute - DEBUG - Writing attribute dataframe to /home/ewan/THOR_output/runs/cpol_demo_geographic/analysis/velocities.csv
2024-09-30 10:09:24,271 - thor.write.attribute - DEBUG - Saving attribute metadata to /home/ewan/THOR_output/runs/cpol_demo_geographic/analysis/velocities.yml


,,u,v,u_ambient,v_ambient,u_shear,v_shear,u_relative,v_relative
time,universal_id,,,,,,,,
2005-11-13 16:00:08,1,-11.4,1.6,-6.8,2.1,-12.4,4.5,-4.6,-0.5
2005-11-13 16:10:23,1,-10.8,1.2,-6.9,2.2,-12.3,4.2,-3.9,-1.1
2005-11-13 16:20:09,1,-10.5,0.9,-7.0,2.4,-12.3,4.0,-3.4,-1.4
2005-11-13 16:30:09,1,-11.0,1.6,-7.1,2.6,-12.0,3.6,-3.9,-1.0
2005-11-13 16:40:09,1,-11.4,1.6,-7.1,2.7,-11.7,3.2,-4.3,-1.1
2005-11-13 16:50:08,1,-10.6,1.6,-7.2,2.9,-11.6,2.9,-3.4,-1.3
2005-11-13 17:00:08,1,-10.6,2.3,-7.0,2.9,-11.4,2.6,-3.6,-0.6
2005-11-13 17:10:23,1,-10.6,2.3,-7.1,2.8,-11.2,2.5,-3.5,-0.5
2005-11-13 17:20:09,1,-10.6,2.3,-7.1,2.7,-11.1,2.1,-3.5,-0.4


In [59]:
analyze.mcs.quality_control(output_directory, analysis_options)

2024-09-30 10:09:25,054 - thor.write.attribute - DEBUG - Writing attribute dataframe to /home/ewan/THOR_output/runs/cpol_demo_geographic/analysis/quality.csv
2024-09-30 10:09:25,057 - thor.write.attribute - DEBUG - Saving attribute metadata to /home/ewan/THOR_output/runs/cpol_demo_geographic/analysis/quality.yml


,,convective_contained,anvil_contained,velocity,shear,relative_velocity,area,offset,major_axis,axis_ratio,duration
time,universal_id,,,,,,,,,,
2005-11-13 16:00:08,1,True,True,True,True,True,True,True,True,True,True
2005-11-13 16:10:23,1,True,True,True,True,True,True,True,True,True,True
2005-11-13 16:20:09,1,True,True,True,True,True,True,True,True,True,True
2005-11-13 16:30:09,1,True,True,True,True,True,True,True,True,True,True
2005-11-13 16:40:09,1,True,False,True,True,True,True,True,True,True,True
2005-11-13 16:50:08,1,True,False,True,True,True,True,True,True,True,True
2005-11-13 17:00:08,1,True,False,True,True,True,True,True,True,True,True
2005-11-13 17:10:23,1,True,False,True,True,True,True,True,True,True,True
2005-11-13 17:20:09,1,True,True,True,True,True,True,True,True,True,True


In [60]:
analyze.mcs.classify(output_directory)

TypeError: classify() missing 2 required positional arguments: 'angles' and 'category_labels'